# File Retrieval

This script downloads required meteorological files from copernicus using the following parameters:
 - Center: Met Office
 - System: 12
 - Variables: 2m Air Temperature, Mean Sea Level Pressure, Total Precipiation
 - Leadtime: 720-1440 hours (24 hour increments)

In [1]:
from tqdm.notebook import tqdm
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
import xarray as xr
import cdsapi
import os

First, , where should we be sending the files? and define the meteorological variables.

In [2]:
output_folder = "E:/31-12-2020/forecast-data/"
meteo_vars = ['2m_temperature', 'mean_sea_level_pressure', 'total_precipitation']

In [3]:
def get_max_min_leadtimes(fmonth, fyear, leadtime):
    """ Determines how many hours are there before a day to be forecasted and how many hours should we be forecasting for.
        Parameters:
            fmonth - Month being forecasted from.
            fyear - Year being forecasted from.
            leadtime - Number of months leadtime to give.
        Returns: Number of hours to first day intended for forecasting and number hours to last day of forecasting (inclusive)."""
    fdate = date(fyear, fmonth, 1)
    ddate = fdate + relativedelta(months=leadtime)
    sdate = ddate + relativedelta(months=1) - relativedelta(days=1)
    min_lt = ((ddate - fdate).days * 24)
    max_lt = ((sdate - fdate).days * 24)
    return min_lt, max_lt

def download_files(meteo_vars, output_folder, leadtime=1):
    """ Downloads the meteorological data in monthly files with the given leadtime. This stores them
        with a file name based on the date of the forecast."""
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    for y in tqdm(range(1993, 2016)):
        for m in range(1, 13):
            if (y == 1993) and (m == 1):
                continue
            file_name = output_folder + "{}-{}.grib".format(m, y)
            # Get the leadtimes
            mi, ma = get_max_min_leadtimes(m, y, leadtime)
            leadtimes = list(range(mi, ma+1, 24))
            if not os.path.exists(file_name):
                print("Downloading {}-{}".format(m, y))
                if y == 1993:
                    download_year(meteo_vars, m, y, leadtimes, file_name)
                else:
                    download_year(meteo_vars, m, y, leadtimes, file_name)
            else:
                print("Already exists: {}-{}".format(m, y))
        
def download_year(meteo_vars, month, year, leadtimes, output_file, system='12'):
    """ Downloads a forecast based on the provided parameters, saving it in the intended file. """
    c = cdsapi.Client()
    c.retrieve(
        'seasonal-original-single-levels',
        {
            'originating_centre': 'ukmo',
            'system': system,
            'variable': meteo_vars,
            'year': year,
            'month': month,
            'day': '01',
            'leadtime_hour': leadtimes,
            'area': [
                70, -100, 10,
                20,
            ],
            'format': 'grib',
        },
        output_file)
    
download_files(meteo_vars, output_folder)

Already exists: 2-1993
Already exists: 3-1993
Already exists: 4-1993
Already exists: 5-1993
Already exists: 6-1993
Already exists: 7-1993
Already exists: 8-1993
Already exists: 9-1993
Already exists: 10-1993
Already exists: 11-1993
Already exists: 12-1993
Already exists: 1-1994
Already exists: 2-1994
Already exists: 3-1994
Already exists: 4-1994
Already exists: 5-1994
Already exists: 6-1994
Already exists: 7-1994
Already exists: 8-1994
Already exists: 9-1994
Already exists: 10-1994
Already exists: 11-1994
Already exists: 12-1994
Already exists: 1-1995
Already exists: 2-1995
Already exists: 3-1995
Already exists: 4-1995
Already exists: 5-1995
Already exists: 6-1995
Already exists: 7-1995
Already exists: 8-1995
Already exists: 9-1995
Already exists: 10-1995


2021-01-17 10:24:30,757 INFO Welcome to the CDS
2021-01-17 10:24:30,758 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-original-single-levels
2021-01-17 10:24:31,109 INFO Request is queued
2021-01-17 10:24:32,138 INFO Request is running


KeyboardInterrupt: 

## Extract Relevant Data

Next, from the forecasted meteorological data we extract the information into files describing the month being forecasted. E.g 01-2020 with a leadtime of 720-1440 will be stored in 02-2020.

In [9]:
def extract_data(meteo_folder, output_folder, mvars=['msl', 't2m', 'tp']):
    """ Extracts the meteorological data from the GRIB files in 'meteo_folder' and stores them in independent files
        named after the dates for the data they represent. E.G file 02-2020 at a leadtime of 1 month represents 03-2020."""
    _, _, files = next(os.walk(meteo_folder))
    data_safety = []
    for f in tqdm(files):
        if f[-5:] == ".grib":
            input_fname = meteo_folder + f
            try:
                ds = xr.open_dataset(input_fname, engine='cfgrib')
                for v in mvars:
                    output_fname = output_folder + v + "/" + f[:-5]
                    if not os.path.exists(output_fname):
                        data = np.mean(ds[v], axis=0)
                        data_safety.append([np.max(data), np.min(data), data.shape, f])
                        np.save(output_fname, data)
            except Exception as e:
                print("Failed to open file: {}".format(input_fname))
                print(e)
            finally:
                ds.close()
    return data_safety
safety = extract_data("E:/31-12-2020/forecast-data/", "E:/31-12-2020/forecastee-data/")

2021-01-17 10:26:31,559 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:26:31,560 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:26:31,572 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:26:31,574 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:26:31,586 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:26:31,587 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:26:32,307 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:26:32,307 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:26:32,329 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:26:32,331 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:26:32,343 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:26:32,344 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:26:33,250 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:26:33,252 INFO missing from GRIB stre

2021-01-17 10:26:46,819 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:26:47,604 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:26:47,605 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:26:47,618 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:26:47,619 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:26:47,631 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:26:47,632 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:26:48,496 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:26:48,497 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:26:48,510 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:26:48,511 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:26:48,522 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:26:48,524 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:26:49,310 INFO missing from GRIB stre

2021-01-17 10:27:05,800 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:05,800 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:06,664 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:06,664 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:06,676 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:06,676 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:06,692 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:06,692 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:07,551 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:07,552 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:07,564 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:07,565 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:07,577 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:07,579 INFO missing from GRIB stre

2021-01-17 10:27:30,877 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:30,892 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:30,892 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:31,879 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:31,879 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:31,920 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:31,920 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:31,941 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:31,941 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:32,901 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:32,901 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:32,943 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:32,959 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:32,965 INFO missing from GRIB stre

2021-01-17 10:27:53,852 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:53,853 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:53,868 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:53,869 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:54,907 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:54,908 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:54,985 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:54,986 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:55,000 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:55,001 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:55,966 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:55,967 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:27:56,018 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:27:56,020 INFO missing from GRIB stre

...

2021-01-17 10:30:03,437 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:03,439 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:03,453 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:03,455 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:04,688 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:04,689 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:04,746 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:04,748 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:04,760 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:04,761 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:05,743 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:05,744 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:05,808 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:05,808 INFO missing from GRIB stre

2021-01-17 10:30:24,306 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:24,365 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:24,366 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:24,380 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:24,381 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:25,349 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:25,350 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:25,425 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:25,427 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:25,440 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:25,442 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:26,414 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:26,415 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:26,466 INFO missing from GRIB stre

2021-01-17 10:30:47,255 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:47,256 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:47,310 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:47,311 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:47,325 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:47,326 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:48,162 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:48,163 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:48,232 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:48,233 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:48,246 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:48,247 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:30:49,416 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:30:49,417 INFO missing from GRIB stre

2021-01-17 10:31:07,540 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:31:08,435 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:31:08,436 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:31:08,474 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:31:08,475 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:31:08,488 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:31:08,489 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:31:09,461 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:31:09,463 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:31:09,515 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:31:09,517 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:31:09,530 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:31:09,531 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:31:10,820 INFO missing from GRIB stre

2021-01-17 10:31:34,395 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:31:34,396 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:31:35,459 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:31:35,460 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:31:35,512 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:31:35,514 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:31:35,529 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:31:35,531 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:31:36,511 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:31:36,512 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:31:36,574 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:31:36,575 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 10:31:36,589 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 10:31:36,590 INFO missing from GRIB stre

In [10]:
def rename_data(data_folder, leadtime=1):
    """ Renames the files to represent the dates for which they actually match. """
    _, _, files = next(os.walk(data_folder))
    for f in tqdm(files):
        date_of_forecast = date(int(f.split('-')[1].split('.')[0]), int(f.split('-')[0]), 1)
        date_forecasted = date_of_forecast + relativedelta(months=1)
        output_file = data_folder + "forecasted-months/{}-{}.npy".format(date_forecasted.month, date_forecasted.year)
        if not os.path.exists(output_file):
            os.rename(data_folder + f, output_file)
rename_data("E:/31-12-2020/forecastee-data/msl/")
rename_data("E:/31-12-2020/forecastee-data/t2m/")
rename_data("E:/31-12-2020/forecastee-data/tp/")